jupyter:
  jupytext:
    formats: ipynb,py:light
    text_representation:
      extension: .py
      format_name: light
      format_version: '1.5'
      jupytext_version: 1.18.1
  kernelspec:
    display_name: Python 3 (ipykernel)
    language: python
    name: python3
---

In [1]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))
from config.spark_config import create_spark

In [2]:
# -----------------------------------------------------
# Project paths
# -----------------------------------------------------
if "__file__" in globals():
    project_root = os.path.abspath(os.path.join(os.path.dirname(__file__), ".."))
else:
    project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

In [3]:
gold_path = os.path.join(project_root, "delta", "gold") + "/"

In [4]:
# -----------------------------------------------------
# Initialize Spark
# -----------------------------------------------------
spark = create_spark("Gold SQL Reporting")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/04 17:20:05 WARN Utils: Your hostname, arber, resolves to a loopback address: 127.0.1.1; using 192.168.1.47 instead (on interface wlp3s0)
25/11/04 17:20:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/arber/lufthansa-data-project/venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/arber/.ivy2.5.2/cache
The jars for the packages stored in: /home/arber/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-94b57591-dd0f-452d-93ca-c954e913f70d;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
:: resolution report :: resolve 154ms :: artifacts dl 5ms
	:: modules

In [5]:
# -----------------------------------------------------
# Load Gold Tables
# -----------------------------------------------------
sales_df = spark.read.format("delta").load(f"{gold_path}kpi_sales_per_category")
delivery_df = spark.read.format("delta").load(f"{gold_path}kpi_avg_delivery_per_seller")
orders_df = spark.read.format("delta").load(f"{gold_path}kpi_orders_per_state")

In [6]:
# Register as temporary SQL views
sales_df.createOrReplaceTempView("kpi_sales_per_category")
delivery_df.createOrReplaceTempView("kpi_avg_delivery_per_seller")
orders_df.createOrReplaceTempView("kpi_orders_per_state")

In [7]:
# -----------------------------------------------------
# 1️⃣ Total Sales per Product Category
# -----------------------------------------------------
print("🔹 Total Sales per Product Category:")
spark.sql("""
    SELECT product_category_name, total_sales
    FROM kpi_sales_per_category
    ORDER BY total_sales DESC
    LIMIT 10
""").show(truncate=False)

🔹 Total Sales per Product Category:


25/11/04 17:20:12 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

+----------------------+------------------+
|product_category_name |total_sales       |
+----------------------+------------------+
|beleza_saude          |1441248.0699999996|
|relogios_presentes    |1305541.6099999994|
|cama_mesa_banho       |1241681.7199999995|
|esporte_lazer         |1156656.4799999988|
|informatica_acessorios|1059272.4000000004|
|moveis_decoracao      |902511.79         |
|utilidades_domesticas |778397.7700000004 |
|cool_stuff            |719329.9500000001 |
|automotivo            |685384.32         |
|ferramentas_jardim    |584219.2099999997 |
+----------------------+------------------+



In [8]:
# -----------------------------------------------------
# 2️⃣ Average Delivery Time per Seller (FIXED)
# -----------------------------------------------------
print("🔹 Average Delivery Time per Seller (Summary):")
spark.sql("""
    SELECT 
        ROUND(MIN(avg_delivery_time), 2) AS min_delivery_time,
        ROUND(PERCENTILE(avg_delivery_time, 0.5), 2) AS median_delivery_time,
        ROUND(AVG(avg_delivery_time), 2) AS mean_delivery_time,
        ROUND(MAX(avg_delivery_time), 2) AS max_delivery_time
    FROM kpi_avg_delivery_per_seller
""").show(truncate=False)

🔹 Average Delivery Time per Seller (Summary):
+-----------------+--------------------+------------------+-----------------+
|min_delivery_time|median_delivery_time|mean_delivery_time|max_delivery_time|
+-----------------+--------------------+------------------+-----------------+
|1.0              |11.0                |11.77             |59.0             |
+-----------------+--------------------+------------------+-----------------+



In [9]:
print("🔹 Sample of Average Delivery Time per Seller:")
spark.sql("""
    SELECT seller_id, avg_delivery_time
    FROM kpi_avg_delivery_per_seller
    ORDER BY avg_delivery_time DESC
    LIMIT 10
""").show(truncate=False)

🔹 Sample of Average Delivery Time per Seller:
+--------------------------------+-----------------+
|seller_id                       |avg_delivery_time|
+--------------------------------+-----------------+
|8629a7efec1aab257e58cda559f03ba7|59.0             |
|3da38366e7bd9baf6369071f782ecdf0|53.0             |
|87f3e35268860433e13d577825aada95|52.0             |
|4e42581f08e8cfc7c090f930bac4552a|47.0             |
|20f0aeea30bc3b8c4420be8ced4226c0|45.0             |
|391bbd13b6452244774beff1824006ed|45.0             |
|ca5832c6960267b71041f74bb39e8b12|45.0             |
|0bb27263628258b8111a0262769fa9db|42.0             |
|6524b847b982cd56bb5d4b02b776ee42|42.0             |
|2a50b7ee5aebecc6fd0ff9784a4747d6|40.0             |
+--------------------------------+-----------------+



In [10]:
# -----------------------------------------------------
# 3️⃣ Number of Orders by State
# -----------------------------------------------------
print("🔹 Number of Orders by State:")
spark.sql("""
    SELECT customer_state, order_count
    FROM kpi_orders_per_state
    ORDER BY order_count DESC
""").show(truncate=False)

🔹 Number of Orders by State:
+--------------+-----------+
|customer_state|order_count|
+--------------+-----------+
|SP            |40495      |
|RJ            |12352      |
|MG            |11355      |
|RS            |5344       |
|PR            |4923       |
|SC            |3547       |
|BA            |3256       |
|DF            |2080       |
|ES            |1995       |
|GO            |1957       |
|PE            |1593       |
|CE            |1279       |
|PA            |946        |
|MT            |886        |
|MA            |717        |
|MS            |701        |
|PB            |517        |
|PI            |476        |
|RN            |474        |
|AL            |397        |
+--------------+-----------+
only showing top 20 rows


In [11]:
spark.stop()
print("✅ Gold SQL reporting completed successfully.")

✅ Gold SQL reporting completed successfully.
